<a href="https://colab.research.google.com/github/carfinkle/carfinkle.github.io/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring First-Year Courses and Outcomes at Whitman College

This notebook will explore the courses and grades taken by Whitman students from fall 2002 through spring 2018. The datasets also contain personal information about each student (gender, race, ethnicity, etc.) and what degree was awarded to each student. This data will be used to examine the 100 level courses taken by specific majors, the grades of students who took a specific class, the average GPA of all majors, and the average GPA of students over time.

## Data Information
The data was presented in two different files. One file contained student grades in their first year courses and the other file contained student information and what degree they graduated with. In order to do this analysis, the two datasets must be merged. They are merged on the `Student ID` column as both datasets have this column in common and this is the variable we are interested in.

In [ ]:
# import the necessary libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#load the necessary files
file_grades = '/content/drive/MyDrive/Senior Year/data science/Data/STEM Data HW 3/Grades-2002-2018_masked.xlsx'
file_outcomes = '/content/drive/MyDrive/Senior Year/data science/Data/STEM Data HW 3/Outcomes-2002-2018_masked.xlsx'

df_grades = pd.read_excel(file_grades)
df_outcomes = pd.read_excel(file_outcomes)

Many students in the datasets lack a major or an end date. This is due to how the data was collected. The data was collected between the 2002 fall semester and the 2018 spring semester. This means that students who were freshman in 2018 most likely have not declared a major and their graduation date (`End Date`) is not included in the dataset. Therefore, we will remove students who either have no major, or no end date.

In [ ]:
#dropna on 'Major 1' and 'End Date'
df_outcomes = df_outcomes.dropna(subset=['Major 1', 'End Date'])

#merge data
df_merge = pd.merge(df_grades, df_outcomes, on="Student ID", how="inner")

## First Year Courses Taken by BBMB Majors

We are interested in seeing the distribution of 100 level classes taken by first years who ended up graduating with a BBMB major. To do so, we first need to filter out the students with a BBMB major. However, there are three columns representing the majors that a student has. Therefore, we must filter through each column to find all students with a BBMB major

In [ ]:
#build a filter that looks for BBMB degrees in any of the three Majors cols
majors = ['BBMB']

def check_major_list(x):
  return (x['Major 1'] in majors) or (x['Major 2'] in majors) or (x['Major 3'] in majors)

df_BBMB = df_merge.apply(check_major_list, axis = 1)

The dataset contains all classes taken by first years. Many first years take classes that are above level 100. However, we are only interested in the 100 level classes (a `Course Number ` of `1xx`). In order to do this, we build a filter that looks for a classes that have a course number of `1xx`. Additionally, we will drop the GENS courses taken by BBMB majors because all first years take this class.

In [ ]:
#build a filter that looks for Course Number 1xx
course_1xx = df_merge['Course Number'] == '1xx'
df_course1xx = df_merge[course_1xx]

#drop GENS subject
df_course1xx = df_course1xx[df_course1xx['Subject'] != 'GENS']

#### Histogram of First Year 100 Level Courses taken by BBMB Majors

In [ ]:
#make a plotly graph_objects histogram of course subject and number 1xx
fig = go.Figure(data = go.Histogram(x = df_course1xx['Subject'][df_BBMB]))
fig.update_layout(title = "100 Level Courses Taken by BBMB Majors",
                  xaxis_title = 'Subject',
                  yaxis_title = '# of Courses')
fig.show()

From the histogram it can be concluded that the 100 level course that BBMB majors took the most their freshman year was chemistry by a large margin. This is logical as General Chemistry is a required 100 level chemistry course for BBMB majors. 100 level math courses are the next most common courses taken by BBMB majors followed by biology

## Grades Earned By BBMB Majors

Many BBMB majors take biology their freshman year. Now we are interested in the grades that these students earned in this class. To do so, the subject column must be filtered to find the `BIOL` classes. Additionally, many students needed to take a biology lab. These classes are classified as `BIOL` but give out zero credits and no grade. Therefore they will be dropped from the dataset.

In [ ]:
#Create a filter that looks for BIOL in Subject
bio_filter = df_course1xx['Subject'] == 'BIOL'
df_bio = df_course1xx[bio_filter]

#remove the bio lab: lab is 0 credits and gives out no grade
df_bio = df_bio[df_bio['Credits'] != 0]

#### Normalized Histogram of The Grades Earned in 100 Level Biology by First Year BBMB Majors

In [ ]:
#Create a normalized histogram of Grade in Biology class for BBMB majors
fig = go.Figure(data = go.Histogram(x = df_bio['Grade'][df_BBMB],
                                    histnorm = 'probability',
                                    xbins=dict(start=0, end=4, size=0.3)))
fig.update_layout(title = "Grades in Biology for BBMB Majors",
                  xaxis_title = 'Grade',
                  yaxis_title = 'Probability')
fig.show()

The histogram reveals that the $25.5\%$ of BBMB majors got a B (3.0 GPA) in their 100 level biology course freshman year. $23.6\%$ of BBMB majors got an A (4.0 GPA), $17.9\%$ got a B+ (3.3 GPA) and $14.2\%$ of students got a A- (3.7 GPA). Overall, the majority of BBMB majors ($81.2\%$) earned a B or higher in their first-year 100 level biology class.

## Grades Earned By Non-BBMB Majors

What about students who are not BBMB majors? We are interested in the grades that these students earned in `BIOL`. To do so, a normalized histogram is created of all non-BBMB majors.

#### Normalized Histogram of The Grades Earned in 100 Level Biology by First Year Non-BBMB Majors

In [ ]:
#Create a normalized histogram of Grade in Biology class for non-BBMB majors
fig = go.Figure(data = go.Histogram(x = df_bio['Grade'][~df_BBMB],
                                    histnorm = 'probability',
                                    xbins=dict(start=0, end=4, size=0.3)))
fig.update_layout(title = "Grades in Biology for Non-BBMB Majors",
                  xaxis_title = 'Grade',
                  yaxis_title = 'Probability')
fig.show()

The histogram reveals that the $20.7\%$ of non-BBMB majors got a B (3.0 GPA) in their 100 level biology course freshman year. $16\%$ of BBMB majors got an A (4.0 GPA), $15.6\%$ got a B+ (3.3 GPA) and $14.6\%$ of students got a A- (3.7 GPA). Overall, the majority of non-BBMB majors ($66.9\%$) earned a B or higher in their first-year 100 level biology class. Therefore, the majority of both groups recieved a grade of a B or higher. However, BBMB majors overall performed better in their first year biology courses than non-BBMB majors.

##Grades Earned by BIOL Majors

What about students who are actually Biology majors? We are interested in the grades that these students earned in `BIOL` `1xx`. To do so, we first need to filter out the students with a `BIOL` major by filtering through each column to find all students with a `BIOL` major. This new dataframe will then be used to create a normalized histogram is created of all Biology majors and their respective grades in 100 level `BIOL` courses.

In [ ]:
majors = ['BIOL']

def check_major_bio(x):
  return (x['Major 1'] in majors) or (x['Major 2'] in majors) or (x['Major 3'] in majors)

df_BIOL = df_merge.apply(check_major_bio, axis = 1)

In [ ]:
#Create a normalized histogram of Grade in Biology class for BBMB majors
fig = go.Figure(data = go.Histogram(x = df_bio['Grade'][df_BIOL],
                                    histnorm = 'probability',
                                    xbins=dict(start=0, end=4, size=0.3)))
fig.update_layout(title = "Grades in Biology for BIOL Majors",
                  xaxis_title = 'Grade',
                  yaxis_title = 'Probability')
fig.show()

Most biology majors ($25.4\%$) earned a B (3.0 GPA) in their first year 100 level biology courses. $16.8\%$ of biology majors earned a B+ (3.3 GPA), $15.1\%$ earned a A- (3.7 GPA), and $18.9\%$ earned an A (4.0 GPA). Overall, $76.2\%$ of biology majors earned a B or higher in their first year, 100 level biology courses. Therefore, it appears that BBMB majors performed the better in this category than biology majors did.

##Average GPA Per Major

With this data, one may assume that we can calculate the most challenging major at Whitman based on grade point average. To do so, we will calculate the GPA of each student and average them based on major.

First we must calculate each student's GPA:

In [ ]:
#calc GPAs
def gpa(x):
  return (x['Grade'] * x['Credits']).sum() / x['Credits'].sum()

GPAs = df_merge.groupby('Student ID').apply(gpa)

<ipython-input-49-6d8b7190bfa4>:5: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Our variable `GPAs` is a series. It cannot be merged with our larger dataset. therefore we must convert the series to a dataframe before merging.

In [ ]:
#Create a dataframe for GPAs
df_gpas= pd.DataFrame(GPAs, columns=['GPA']) #no column name = 0

# merge df_gpas with with df_merge on "Student ID" (defualt inner)
df_merge_gpa = pd.merge(df_merge, df_gpas, on="Student ID", how="inner")

In order to average each student's GPA based on major, we must put all three major columns into one column. We use the `pd.melt()` function to accomplish this.

In [ ]:
# melt 'Major 1', 'Major 2', and 'Major 3' into one column
df_all_majors = pd.melt(df_merge_gpa, id_vars = ['Student ID', 'GPA'],
                        value_vars = ['Major 1', 'Major 2', 'Major 3'],
                        var_name = 'Major Type', value_name = 'Major')

#drop rows where Major is NaN
df_all_majors = df_all_majors.dropna(subset=['Major'])
df_all_majors.head()

,Student ID,GPA,Major Type,Major
0,356167,3.485714,Major 1,GE.AS
1,356167,3.485714,Major 1,GE.AS
2,356167,3.485714,Major 1,GE.AS
3,356167,3.485714,Major 1,GE.AS
4,356167,3.485714,Major 1,GE.AS


In [ ]:
#Compute average GPA per major
grade = df_all_majors.groupby('Major')['GPA'].mean()

df_major_gpa = pd.DataFrame(grade)
df_major_gpa.columns = ['Average GPA'] #rename column title

####Bar Graph of Major vs GPA of Freshman Courses

In [ ]:
#graph Major on the x axis and average GPA on the y
my_scale = ['rgb(255,0,0)', 'rgb(3,252,40)']

fig = px.bar(x = df_major_gpa.index, y = df_major_gpa['Average GPA'],
             color=df_major_gpa['Average GPA'], color_continuous_scale=my_scale)
fig.update_yaxes(range = [2, 4])
fig.update_layout(title = "Average GPA per Major for Freshman (2002 - 2018)",
                  xaxis_title = 'Major', yaxis_title = 'Average GPA')

fig.show()

Here it shows that the major with the lowest average freshman GPA was philisophical creative writing and the major with the highest average freshman GPA was international health. One may conclude that philisophical creative writing is the hardest major and international health is the easiest major. However, we cannot make this conclusion. This graph does not reduce each major to only include grades from their respective required courses. Freshman, moreso than other classes, probably take more distribution classes. Therefore, we can only conclude that philisophical creative writing had the lowest average first year GPA between 2002 and 2018 and international health had the highest.

##Average GPA Per Math Major

Which type of math major had the highest average freshmen GPA? Which had the lowest? To answer this question we must first filter through the `Major` column to pull out only math/math-combined majors. We then add the resulting dataFrame (converted from a series) to our `df_all_majors` dataFrame.



In [ ]:
#build a filter that looks for selected degrees the Major Column
majors = ['MATH', 'EC.MA', 'MA.PH', 'MA.ST', 'MA.PC']

def check_major_list(x):
  return (x['Major'] in majors)

df_math = df_all_majors.apply(check_major_list, axis = 1)

In [ ]:
#dataFrame of GPA of math majors
df_math_gpa = df_all_majors[df_math]

#Compute average GPA per major
grade_math = df_math_gpa.groupby('Major')['GPA'].mean()

df_math_gpa = pd.DataFrame(grade_math)
df_math_gpa.columns = ['Average GPA'] #rename GPA column

####Bar Graph of Math Major vs GPA of Freshman Courses

In [ ]:
fig = go.Figure(data = go.Bar(x = df_math_gpa.index, y = df_math_gpa['Average GPA']))
fig.update_yaxes(range = [2.5, 4])
fig.update_layout(title = "Average GPA per Math Major for Freshman (2002 - 2018)",
                  xaxis_title = 'Math Major', yaxis_title = 'Average GPA')
fig.show()

We therefore conclude that on average, out of the math majors, Math Physics majors had the lowest average first year GPA and pure math majors had the highest average first year GPA. Econ/Math majors came in second.

##Above and Beyond

In this section, we will explore how student's GPA changed over time. In order to do so, we must create a new column that holds the date of the terms instead of using the notation 2002FA, 2003SP, etc. First we will drop the columns we are not using in order to better visualize our data. Next we will build a filter that examines the `Term` column of the dataset. If the term is a fall term, it will return the term year and date 8/30 (rough Whitman start date for the fall semester). If the term is a spring term, it will return the term year and date 5/15 (rough Whitman end date for the spring semester).

In [ ]:
# drop columns: "Academic Division", 'Subject', 'Course Number', 'Credits',
#'Grade', 'Start Term', 'End Date'
df_dropped = df_merge_gpa.drop(columns = ['Academic Division', 'Subject',
                                          'Course Number', 'Credits', 'Grade',
                                          'Start Term', 'End Date'])

#build a filter that looks at Term and returns a date
def convert_term_to_date(x):
  year = x[:4] #extract the year
  if x[4:] == 'FA': #if it's fall
    return f'{year}-08-30'
  elif x[4:] == 'SP': #if it's spring
    return f'{year}-05-15'
  else:
    return None

df_dropped['Term Date'] = df_dropped['Term'].apply(convert_term_to_date)
df_dropped.head()


,Student ID,Term,Gender,Races,Ethnicity,First Generation,Major 1,Major 2,Major 3,GPA,Term Date
0,356167,2002FA,F,WH,NHS,N,GE.AS,NaN,NaN,3.485714,2002-08-30
1,356167,2002FA,F,WH,NHS,N,GE.AS,NaN,NaN,3.485714,2002-08-30
2,356167,2002FA,F,WH,NHS,N,GE.AS,NaN,NaN,3.485714,2002-08-30
3,356167,2002FA,F,WH,NHS,N,GE.AS,NaN,NaN,3.485714,2002-08-30
4,356167,2002FA,F,WH,NHS,N,GE.AS,NaN,NaN,3.485714,2002-08-30


Next we need to find the average GPA per term of all the students in our dataset.

In [ ]:
#Average GPA per term for all students.
gpa_over_time = df_dropped.groupby('Term Date')['GPA'].mean()
df_gpa_over_time = pd.DataFrame(gpa_over_time)
df_gpa_over_time.columns = ['Average GPA']
df_gpa_over_time.head()

,Average GPA
Term Date,
2002-08-30,3.267158
2003-05-15,3.265444
2003-08-30,3.299362
2004-05-15,3.301609
2004-08-30,3.299586


Now that we have the average GPA per term of all the students in the dataset we can compare the GPAs overtime between male and female identifying students. As well as compare the average GPA overtime between first generation and non-first gen students.

In [ ]:
#Average GPA over time for men and women
gb_gender = df_dropped.groupby(['Gender', 'Term Date'])['GPA'].mean()
df_gender = pd.DataFrame(gb_gender)
df_gender.columns = ['Average GPA']
df_gender.head()

Average GPA
Gender Term Date              
F      2002-08-30     3.344967
       2003-05-15     3.342489
       2003-08-30     3.341868
       2004-05-15     3.339113
       2004-08-30     3.373730

In [ ]:
#Average GPA over time for first generation students
gb_firstgen = df_dropped.groupby(['First Generation', 'Term Date'])['GPA'].mean()
df_firstgen = pd.DataFrame(gb_firstgen)
df_firstgen.columns = ['Average GPA']
df_firstgen.head()

Average GPA
First Generation Term Date              
N                2002-08-30     3.267158
                 2003-05-15     3.265444
                 2003-08-30     3.299362
                 2004-05-15     3.301609
                 2004-08-30     3.299586

#### Line Graph of GPAs Overtime of Whitman Freshman (Men vs Women)

In [ ]:
#Create a plot of average GPA over time
fig = go.Figure(data = go.Scatter(x = df_gpa_over_time.index,
                                  y = df_gpa_over_time['Average GPA'],
                                  name = 'All',
                                  line = dict(color = 'green')))

#layer a plot of average GPA for M over time
fig.add_trace(go.Scatter(x = df_gender.loc['M'].index,
                         y = df_gender.loc['M']['Average GPA'],
                         name = 'Men',
                         line = dict(color = 'blue')))

#layer a plot of average GPA for F over time
fig.add_trace(go.Scatter(x = df_gender.loc['F'].index,
                         y = df_gender.loc['F']['Average GPA'],
                         name = 'Women',
                         line = dict(color = 'pink')))

fig.update_layout(title = "Average GPA over Time",
                  xaxis_title = 'Term', yaxis_title = 'Average GPA')

date_buttons = [
    {'count': 9, 'label': '2015FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 21, 'label': '2014FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 33, 'label': '2013FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 45, 'label': '2012FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 57, 'label': '2011FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 69, 'label': '2010FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 81, 'label': '2009FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 93, 'label': '2008FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 105, 'label': '2007FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 117, 'label': '2006FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 129, 'label': '2005FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 141, 'label': '2004FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 153, 'label': '2003FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
]

# Add date buttons
fig.update_layout(
    {'xaxis':
    {'rangeselector' : {'buttons' : date_buttons}}})

# Add a range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

From the graph we see that those who identify as female tended to have a higher average GPA then men. This trend is present until 2016 where male GPA spiked to 3.85. Interestingly, there was a large dip in both groups during the fall semester of 2015.

#### Line Graph of GPAs Overtime of Whitman Freshman (First Gen vs Non-First Gen)

In [ ]:
#Create a plot of average GPA over time
fig = go.Figure(data = go.Scatter(x = df_gpa_over_time.index,
                                  y = df_gpa_over_time['Average GPA'],
                                  name = 'All',
                                  line = dict(color = 'green')))

#layer a plot of average GPA for N over time
fig.add_trace(go.Scatter(x = df_firstgen.loc['N'].index,
                         y = df_firstgen.loc['N']['Average GPA'],
                         name = 'Not First Gen',
                         line = dict(color = 'orange')))

#layer a plot of average GPA for Y over time
fig.add_trace(go.Scatter(x = df_firstgen.loc['Y'].index,
                         y = df_firstgen.loc['Y']['Average GPA'],
                         name = 'First Gen',
                         line = dict(color = 'yellow')))

fig.update_layout(title = "Average GPA over Time (First Generation Students)",
                  xaxis_title = 'Term', yaxis_title = 'Average GPA')

date_buttons = [
    {'count': 9, 'label': '2015FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 21, 'label': '2014FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 33, 'label': '2013FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 45, 'label': '2012FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 57, 'label': '2011FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 69, 'label': '2010FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 81, 'label': '2009FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 93, 'label': '2008FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 105, 'label': '2007FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 117, 'label': '2006FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 129, 'label': '2005FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 141, 'label': '2004FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
    {'count': 153, 'label': '2003FA-2016SP', 'step': 'month', 'stepmode': 'todate'},
]

# Add date buttons
fig.update_layout(
    {'xaxis':
    {'rangeselector' : {'buttons' : date_buttons}}})

# Add a range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

From this graph we can see that either Whitman did not start tracking whether or not a student was a first generation student until 2005, or there were no first generation students in this dataset until 2005. Most likely it's the former. We can see that on average non-first generation students had higher GPAs on average until August 2014 in which they were nearly identical. We again see a massive dip in both groups' average GPA in 2015. Finally, in 2016, the average GPAs of first generation students appears to pass non-first gen students.